In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.pandas.set_option("display.max_columns",None)
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling

## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
df=pd.read_csv(('raw_data.csv'))
        

In [15]:
df = df.drop(labels=["ID","Delivery_person_ID","Order_Date","Type_of_order"],axis=1)
df["Time_order_hour"] = df["Time_Orderd"].str.split(":").str[0]
df["Time_order_min"] = df["Time_Orderd"].str.split(":").str[1]
df = df.drop(labels=["Time_Orderd"],axis=1)
df["Time_order_picked_hour"] = df["Time_Order_picked"].str.split(":").str[0]
df["Time_order_picked_min"] = df["Time_Order_picked"].str.split(":").str[1]
df = df.drop(labels=["Time_Order_picked"],axis=1)
df["Time_order_hour"] = df["Time_order_hour"].astype(float)
df["Time_order_min"] = df["Time_order_min"].astype(float)
df["Time_order_picked_hour"] = df["Time_order_picked_hour"].astype(float)
df["Time_order_picked_min"] = df["Time_order_picked_min"].astype(float)
df1 = df.copy()
numerical_columns=df.columns[df.dtypes!='object']
categorical_columns=df.columns[df.dtypes=='object']
df1 = df1[numerical_columns]
df2 = df1.fillna(value = df1.median())
df1 = df[categorical_columns]
df2["Weather_conditions"] = df1["Weather_conditions"].fillna(method = 'ffill')
df2["Road_traffic_density"] = df1["Road_traffic_density"].fillna(value = 'Medium')
df2["Festival"] = df1["Festival"].fillna(value = 'No')
df2["City"] = df1["City"].fillna(value = 'Semi-Urban')
x = df2.drop(labels=["Time_taken (min)"],axis=1)
y = df2[["Time_taken (min)"]]
df2 = x
df2['Time_taken (min)'] = y
df = pd.DataFrame(df2)


In [25]:
df.head(2)

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_order_hour,Time_order_min,Time_order_picked_hour,Time_order_picked_min,Weather_conditions,Road_traffic_density,Festival,City,Time_taken (min)
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,2,3.0,21.0,55.0,22.0,10.0,Fog,Jam,No,Metropolitian,46
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,1,1.0,14.0,55.0,15.0,5.0,Stormy,High,No,Metropolitian,23


In [26]:
df['Delivery_person_Age'].unique()

array([36., 21., 23., 34., 24., 29., 35., 33., 25., 31., 37., 27., 32.,
       26., 38., 20., 22., 30., 28., 39., 15., 50.])

In [27]:
df['Delivery_person_Ratings'].unique()

array([4.2, 4.7, 4.3, 4.5, 4. , 4.9, 4.1, 5. , 4.8, 3.5, 4.6, 4.4, 3.8,
       3.9, 3.7, 2.6, 2.5, 3.6, 3.1, 2.7, 1. , 3.2, 3.3, 6. , 3.4, 2.8,
       2.9, 3. ])

In [37]:
df['Delivery_location_latitude'].unique()

array([30.397968, 10.043064, 18.65245 , ..., 30.982872, 15.636205,
       20.005337])

In [38]:
df['Time_order_picked_hour'].unique()

array([22.        , 15.        , 17.        ,  9.        , 20.        ,
       21.        ,  8.        , 19.        , 12.        , 18.        ,
       23.        , 13.        , 14.        , 11.        ,  0.45833333,
       16.        ,  0.75      ,  0.95833333,  0.91666667, 10.        ,
        0.875     , 24.        ,  0.375     ,  0.83333333,  0.70833333,
        0.625     ,  1.        ,  0.79166667,  0.5       ,  0.54166667,
        0.41666667,  0.58333333,  0.66666667])

In [32]:
df['Vehicle_condition'].unique()

array([2, 1, 0, 3], dtype=int64)

In [33]:
df['multiple_deliveries'].unique()

array([3., 1., 0., 2.])

In [36]:
df['Weather_conditions'].unique()

array(['Fog', 'Stormy', 'Sandstorms', 'Windy', 'Cloudy', 'Sunny'],
      dtype=object)

In [35]:
df['Delivery_person_Age'].unique()

array([36., 21., 23., 34., 24., 29., 35., 33., 25., 31., 37., 27., 32.,
       26., 38., 20., 22., 30., 28., 39., 15., 50.])

In [39]:
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Vehicle_condition,multiple_deliveries,Time_order_hour,Time_order_min,Time_order_picked_hour,Time_order_picked_min,Weather_conditions,Road_traffic_density,Festival,City,Time_taken (min)
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,2,3.0,21.0,55.0,22.0,10.0,Fog,Jam,No,Metropolitian,46
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,1,1.0,14.0,55.0,15.0,5.0,Stormy,High,No,Metropolitian,23
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,1,1.0,17.0,30.0,17.0,40.0,Sandstorms,Medium,No,Metropolitian,21
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,0,0.0,9.0,20.0,9.0,30.0,Sandstorms,Low,No,Metropolitian,20
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,1,1.0,19.0,50.0,20.0,5.0,Fog,Jam,No,Metropolitian,41
